# Sarcasm Detection
 **Acknowledgement**

Misra, Rishabh, and Prahal Arora. "Sarcasm Detection using Hybrid Neural Network." arXiv preprint arXiv:1908.07414 (2019).

**Required Files given in below link.**

https://drive.google.com/drive/folders/1xUnF35naPGU63xwRDVGc-DkZ3M8V5mMk

## Install `Tensorflow2.0` 

In [ ]:
!!pip uninstall tensorflow
!pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 50kB/s 
     |████████████████████████████████| 3.8MB 37.0MB/s 
     |████████████████████████████████| 450kB 29.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=1a51766f2df465b73194c73d2e5dcf6e97eb3c0ba40cd527ead709b504b5dca0
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


## Get Required Files from Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
#Set your project path 
project_path =  "/content/drive/My Drive/Sequence Model-NLP/Assignment - Sarcasm detection/"

#**## Reading and Exploring Data**

## Read Data "Sarcasm_Headlines_Dataset.json". Explore the data and get  some insights about the data. ( 4 marks)
Hint - As its in json format you need to use pandas.read_json function. Give paraemeter lines = True.

In [24]:
import pandas as pd
df = pd.read_json(project_path + "Sarcasm_Headlines_Dataset.json", lines=True)

df.head(10)

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
5,https://www.huffingtonpost.com/entry/advancing...,advancing the world's women,0
6,https://www.huffingtonpost.com/entry/how-meat-...,the fascinating case for eating lab-grown meat,0
7,https://www.huffingtonpost.com/entry/boxed-col...,"this ceo will send your kids to school, if you...",0
8,https://politics.theonion.com/top-snake-handle...,top snake handler leaves sinking huckabee camp...,1
9,https://www.huffingtonpost.com/entry/fridays-m...,friday's morning email: inside trump's presser...,0


## Drop `article_link` from dataset. ( 2 marks)
As we only need headline text data and is_sarcastic column for this project. We can drop artical link column here.

In [25]:
df.drop(['article_link'], axis=1, inplace= True)
#Article links doesnot help derive any insights for our usecase

## Get the Length of each line and find the maximum length. ( 4 marks)
As different lines are of different length. We need to pad the our sequences using the max length.

In [26]:
df.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [27]:
df['lengths'] = df['headline'].apply(lambda x: len(x.split(" ")))
df.head()
#Above will calculate the number of words in each of the headlines

,headline,is_sarcastic,lengths
0,former versace store clerk sues over secret 'b...,0,12
1,the 'roseanne' revival catches up to our thorn...,0,14
2,mom starting to fear son's web series closest ...,1,14
3,"boehner just wants wife to listen, not come up...",1,13
4,j.k. rowling wishes snape happy birthday in th...,0,11


#**## Modelling**

## Import required modules required for modelling.

In [28]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential

# Set Different Parameters for the model. ( 2 marks)

In [31]:
df['lengths'].max()

39

In [34]:
max_features = 10000
maxlen = 30 # despise the max len of 39 words, considering 30 most frequent words taken into account 
embedding_size = 200

## Apply Keras Tokenizer of headline column of your data.  ( 4 marks)
Hint - First create a tokenizer instance using Tokenizer(num_words=max_features) 
And then fit this tokenizer instance on your data column df['headline'] using .fit_on_texts()

In [35]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(df['headline']))

# Define X and y for your model.

In [37]:
'''texts_to_sequences is used to transforms each text in texts to a sequence of integers. Only top num_words-1 most frequent words will 
be taken into account. Only words known by the tokenizer will be taken into account.'''
X = tokenizer.texts_to_sequences(df['headline'])
# Pad_sequences used to ensure that all sequences in a list have the same length
X = pad_sequences(X, maxlen = maxlen)
y = np.asarray(df['is_sarcastic'])

print("Number of Samples:", len(X))
print(X[0])
print("Number of Labels: ", len(y))
print(y[0])

Number of Samples: 26709
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0  307  678 3336 2297   47  381 2575    5
 2576 8433]
Number of Labels:  26709
0


## Get the Vocabulary size ( 2 marks)
Hint : You can use tokenizer.word_index.

In [39]:
numOfWords = len(tokenizer.word_index) + 1
print(numOfWords) # total size of the words we are dealing with in this use case

29657


#**## Word Embedding**

## Get Glove Word Embeddings

In [40]:
glove_file = project_path + "glove.6B.zip"

In [41]:
#Extract Glove embedding zip file
from zipfile import ZipFile
with ZipFile(glove_file, 'r') as z:
  z.extractall()

# Get the Word Embeddings using Embedding file as given below.

In [42]:
EMBEDDING_FILE = './glove.6B.200d.txt'

embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    embeddings[word] = embd



# Create a weight matrix for words in training docs

In [44]:
embedding_matrix = np.zeros((numOfWords, 200))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

len(embeddings.values())

400000

## Create and Compile your Model  ( 7 marks)
Hint - Use Sequential model instance and then add Embedding layer, Bidirectional(LSTM) layer, then dense and dropout layers as required. 
In the end add a final dense layer with sigmoid activation for binary classification.


In [46]:
model = Sequential()
model.add(Embedding(numOfWords, embedding_size, weights = [embedding_matrix]))

model.add(Bidirectional(LSTM(128, return_sequences = True)))

model.add(Dense(40, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit your model with a batch size of 100 and validation_split = 0.2. and state the validation accuracy ( 5 marks)


In [48]:
batch_size = 100
epochs = 15

## Add your code here ##
history = model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/15
214/214 [==============================] - 17s 81ms/step - loss: 0.0606 - accuracy: 0.9798 - val_loss: 0.7930 - val_accuracy: 0.8521
Epoch 2/15
214/214 [==============================] - 17s 80ms/step - loss: 0.0383 - accuracy: 0.9886 - val_loss: 1.0900 - val_accuracy: 0.8531
Epoch 3/15
214/214 [==============================] - 17s 80ms/step - loss: 0.0283 - accuracy: 0.9910 - val_loss: 1.2304 - val_accuracy: 0.8528
Epoch 4/15
214/214 [==============================] - 17s 79ms/step - loss: 0.0298 - accuracy: 0.9901 - val_loss: 1.2446 - val_accuracy: 0.8493
Epoch 5/15
214/214 [==============================] - 17s 80ms/step - loss: 0.0176 - accuracy: 0.9941 - val_loss: 1.4183 - val_accuracy: 0.8508
Epoch 6/15
214/214 [==============================] - 17s 80ms/step - loss: 0.0120 - accuracy: 0.9965 - val_loss: 1.5858 - val_accuracy: 0.8494
Epoch 7/15
214/214 [==============================] - 17s 79ms/step - loss: 0.0128 - accuracy: 0.9962 - val_loss: 1.5091 - val_accuracy: